# Utilities

## Data Preparation

In [1]:
# Function to list all .docx files in a given folder
def list_docx_files(folder_path, file_extension=".docx"):
    from pathlib import Path
    import docx

    path = Path(folder_path)
    # Using glob to find files that match the pattern *[file_extension]
    return [file.name for file in path.glob(f"*{file_extension}") if file.is_file()]


# Function to read a .docx file, join paragraph texts, and split it by "****"
def split_docx(file_path):
    import docx

    doc = docx.Document(file_path)
    # Join all paragraphs with newline character
    full_text = "\n".join([para.text for para in doc.paragraphs])
    # Split the full text by the separator "****"
    return full_text.split("****")

## List Subdirectories

In [2]:
def get_subfolder_paths(folder_path):
    import os

    subfolder_paths = []

    # Check if the given path exists and is a directory
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        # Get the full path of the folder
        full_path = os.path.abspath(folder_path)

        # Get the last two parts of the folder path
        path_parts = full_path.split(os.sep)
        parent_path = path_parts[-1]
        print(parent_path)

        # Iterate through all items in the directory
        for item in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item)
            # If the item is a directory, add its path to the list
            if os.path.isdir(item_path):
                subfolder_path = f"./{parent_path}/{item}"
                subfolder_paths.append(subfolder_path)

    return subfolder_paths

## Json

In [3]:
def save_embeddings_documents_to_json(data, folder_path, filename):
    import os
    import json

    # Ensure the folder exists
    os.makedirs(folder_path, exist_ok=True)

    # Construct the full file path
    file_path = os.path.join(folder_path, filename)

    # Write the data to a JSON file
    with open(file_path, "w") as json_file:
        json.dump(data, json_file, indent=4)

    print(f"JSON file saved successfully at: {file_path}")

In [4]:
def load_json_to_mongodb_docs(file_path):
    import json

    """
    Load JSON data from a file into a variable named docs.

    :param file_path: Path to the JSON file
    :return: The loaded JSON data
    """
    try:
        with open(file_path, "r") as json_file:
            docs = json.load(json_file)
        print(f"JSON file loaded successfully from: {file_path}")
        return docs
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file at {file_path}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return None


## MongoDB

In [5]:
def mongodb_init():
    import os
    from dotenv import load_dotenv
    from pymongo import MongoClient

    load_dotenv()

    MONGODB_CONNECTION_STRING: str | None = os.getenv(key="MONGODB_CONNECTION_STRING", default=None)

    client = MongoClient(
        host=MONGODB_CONNECTION_STRING,
        tls=True,
        tlsAllowInvalidCertificates=True,
    )
    print(f"MongoDB client: {client}")
    return client

## OpenAI

In [6]:
def embeddingmodel_init():
    import os
    from dotenv import load_dotenv
    from langchain_openai import OpenAIEmbeddings

    load_dotenv()

    OPENAI_API_KEY: str | None = os.getenv(key="OPENAI_API_KEY", default=None)

    embedding_model = OpenAIEmbeddings(
        model="text-embedding-3-large",
        dimensions=3072,
        disallowed_special=(),
        api_key=OPENAI_API_KEY,
    )
    print(f"Embedding model: {embedding_model}")
    return embedding_model


# Data Preparation Pipeline

In [7]:
def data_preparation_pipeline(paper_name, folder_path, file_extension=".docx"):
    files: list[str] = list_docx_files(
        folder_path=folder_path,
        file_extension=file_extension,
    )
    files = [f"{folder_path}/{file}" for file in files]
    print("List of .docx files:", len(files), files)
    print("*" * 100)
    multiple_sections: list[tuple[str, str, str, list[str]]] = []
    for file in files:
        # Split the document into sections using the defined function
        sections: list[str] = split_docx(file_path=file)
        # Remove any leading/trailing whitespace from each section
        sections = [section.strip() for section in sections if len(section) > 1]
        chapter_name: str = sections[0]
        unit_name: str = sections[1]
        sections.remove(chapter_name)
        sections.remove(unit_name)

        # Debug prints to check the number of sections and their content)
        print("List of sections per file:", file, len(sections), sections)
        print("Paper:", paper_name)
        print("Chapter:", chapter_name)
        multiple_sections.append((
            paper_name,
            chapter_name,
            unit_name,
            sections,
        ))
        print("*" * 100)
    print(len(multiple_sections), multiple_sections[0])
    return multiple_sections

# Pipeline: Convert Embeddings to MongoDB Documents

In [8]:
def mongodb_document_with_embedding_pipeline(
    paper_name: str,
    chapter_name: str,
    unit_name: str,
    sections: list[str],
    embeddings: list[list[float]],
):
    docs: list[dict] = []
    for i in range(len(sections)):
        docs.append({
            "paper": paper_name,
            "chapter": chapter_name,
            "unit": unit_name,
            "text": sections[i],
            "embedding": embeddings[i],
        })
    print(len(docs), docs[0]["unit"])
    return docs

# Pipeline: Save Embeddings Documents to Json

In [9]:
def save_embeddings_documents_to_json_pipeline(
    paper_name="CA-FO-P1",
    folder_path="./ca-fo-p1/Chapter 1",
    destination_path="./generated_embeddings",
):
    # data:list[tuple[paper, chapter, unit , sections]]
    data: list[tuple[str, str, list[str]]] = data_preparation_pipeline(
        paper_name=paper_name,
        folder_path=folder_path,
        file_extension=".docx",
    )
    embedding_model = embeddingmodel_init()
    for datum in data:
        paper_name = datum[0]
        chapter_name = datum[1]
        unit_name = datum[2]
        sections = datum[3]
        embeddings = embedding_model.embed_documents(texts=sections)
        docs = mongodb_document_with_embedding_pipeline(
            paper_name=paper_name,
            chapter_name=unit_name,
            unit_name=unit_name,
            sections=sections,
            embeddings=embeddings,
        )
        save_embeddings_documents_to_json(
            data=docs,
            folder_path=destination_path,
            filename=f"{unit_name}.json",
        )

In [64]:
%%time
save_embeddings_documents_to_json_pipeline(
    paper_name="CA-FO-P4",
    folder_path="./CA-FO-P4/Chapter 10",
    destination_path="./generated_embeddings/CA-FO-P4",
)

List of .docx files: 1 ['./CA-FO-P4/Chapter 10/CA-FO-P4-C10 - Praveen.docx']
****************************************************************************************************
List of sections per file: ./CA-FO-P4/Chapter 10/CA-FO-P4-C10 - Praveen.docx 18 ["Introduction of Status of Indian Economy: Pre Independence Period (1850-1947)\nBetween the first and the seventeenth century AD, India is believed to have had the largest economy of the ancient and the medieval world. It was prosperous and self-reliant and is believed to have controlled between one third and one fourth of the world's wealth. The economy consisted of self-sufficient villages as well as cities which were centres of commerce, pilgrimage and administration. Compared to villages, cities presented more opportunities for diverse occupations, trades and gainful economic activities.\nSimple division of labour intertwined with attributes such as race, class, and gender was the basis of the structure of the villages and acte

In [ ]:
# %%time
# chapters = get_subfolder_paths(folder_path="./ca-fo-p1")
# chapters.remove("./ca-fo-p1/Chapter 1")

# for chapter in chapters:
#     print(f"Chapter: {chapter}")
#     save_embeddings_documents_to_json_pipeline(
#         paper_name="CA-FO-P1",
#         folder_path=chapter,
#     )

ca-fo-p1
Chapter: ./ca-fo-p1/Chapter 7
List of .docx files: 2 ['./ca-fo-p1/Chapter 7/CA-FO-P1-C7-U1-Ashok.docx', './ca-fo-p1/Chapter 7/CA-FO-P1-C7-U2-Ashok.docx']
****************************************************************************************************
List of sections per file: ./ca-fo-p1/Chapter 7/CA-FO-P1-C7-U1-Ashok.docx 32 ["Unit Overview: Final Accounts of Non-Manufacturing Business Entities\nIn the context of non-manufacturing business entities, the final accounts are crucial for determining the financial performance and position at the end of an accounting period. These accounts typically include:\nTrading Account: This account is used to calculate the gross profit or loss of the business. It summarizes the costs of goods sold and the revenue from sales. The primary purpose of the trading account is to ascertain the effectiveness of the trading activities of the entity.\nProfit & Loss Account: Following the trading account, the profit & loss account is prepared to de

# Pipeline: MongoDB Data Ingestion

In [22]:
client = mongodb_init()
client.list_database_names()

MongoDB client: MongoClient(host=['mmdev-shard-00-02.z7q8g.mongodb.net:27017', 'mmdev-shard-00-01.z7q8g.mongodb.net:27017', 'mmdev-shard-00-00.z7q8g.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-b4um4g-shard-0', tls=True, tlsallowinvalidcertificates=True, tlsdisableocspendpointcheck=True)


['Education',
 'ca_foundation',
 'ecommerce',
 'embeddings',
 'meramaster',
 'mm_ai',
 'mmdev',
 'mobileapp',
 'subrata',
 'tanuja',
 'your_database_name',
 'admin',
 'local']

In [50]:
paper_name = "CA-FO-P1"  # This is the collection name, so keep it same for all the different papers
db = client.get_database(name="ca_foundation")
coll = db.get_collection(name=paper_name)
coll

Collection(Database(MongoClient(host=['mmdev-shard-00-02.z7q8g.mongodb.net:27017', 'mmdev-shard-00-01.z7q8g.mongodb.net:27017', 'mmdev-shard-00-00.z7q8g.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-b4um4g-shard-0', tls=True, tlsallowinvalidcertificates=True, tlsdisableocspendpointcheck=True), 'ca_foundation'), 'CA-FO-P1')

In [65]:
json_files = list_docx_files(folder_path="./generated_embeddings/CA-FO-P4", file_extension=".json")
print(len(json_files), json_files)

25 ['CA-FO-P4-C9-U4 Exchange Rate and Its Economics Effects.json', 'CA-FO-P4-C10 Indian Economy.json', 'CA-FO-P4-C2-U3-SUPPLY.json', 'CA-FO-P4-C3-U1-THEORY OF PRODUCTION AND COST.json', 'CA-FO-P4-C9-U5 International Capital Movements.json', 'CA-FO-P3-C9-U3 Trade Negotiations.json', 'CA-FO-P3-C8-U2 Monetary Policy.json', 'CA-FO-P4-C5-U-BUSINESS CYCLES.json', 'CA-FO-P4-C2-U1-THEORY OF CONSUMER BEHAVIOUR.json', 'CA-FO-P3-C9-U2 The Instruments of Trade Policy.json', 'CA-FO-P4-C7-U3 The Process of Budget making: Sources of Revenue, Expenditure management and Management of Public Debt.json', 'CA-FO-P3-C8-U1 The Concept of Money Demand: Important Theories.json', 'CA-FO-P4-C2-U1=THEORY OF DEMAND AND SUPPLY.json', 'CA-FO-P4-C4-U2-DETERMINATION OF PRICES.json', 'CA-FO-P4-C7-U4 Fiscal Policy.json', 'CA-FO-P4-C1-U2-BASIC PROBLEMS OF AN ECONOMY AND ROLE OF PRICE MECHANISM.json', 'CA-FO-P3-C8-U2 Concept of money Supply.json', 'CA-FO-P3-C9-U1 Theories International Trade.json', 'CA-FO-P4-C1-U1-Introd

In [26]:
def mongodb_embeddings_ingestion_pipeline(
    db_name: str,
    paper_name: str,
    json_folder_path: str,
):
    client = mongodb_init()
    db = client.get_database(name=db_name)
    coll = db.get_collection(name=paper_name)
    print(f"Collection: {coll}")
    print(f"Total Documents before Ingestion: {coll.count_documents(filter={})}")

    json_files = list_docx_files(folder_path=json_folder_path, file_extension=".json")
    print(f"Total Documents to be Ingested: {len(json_files)}")
    for file in json_files:
        docs = load_json_to_mongodb_docs(file_path=f"{json_folder_path}/{file}")
        try:
            coll.insert_many(documents=docs)
            print(f"{len(docs)} Documents inserted successfully")
        except Exception as e:
            print(f"An error occurred during insertion: {e}")

        print("Documents embedded and inserted successfully")
    print(f"Total Documents after Ingestion: {coll.count_documents(filter={})}")

In [66]:
%%time
mongodb_embeddings_ingestion_pipeline(
    db_name="ca_foundation",
    paper_name="CA-FO-P1",  # Colection name so keep it same for all different papers
    json_folder_path="./generated_embeddings/CA-FO-P4/",
)

MongoDB client: MongoClient(host=['mmdev-shard-00-02.z7q8g.mongodb.net:27017', 'mmdev-shard-00-01.z7q8g.mongodb.net:27017', 'mmdev-shard-00-00.z7q8g.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-b4um4g-shard-0', tls=True, tlsallowinvalidcertificates=True, tlsdisableocspendpointcheck=True)
Collection: Collection(Database(MongoClient(host=['mmdev-shard-00-02.z7q8g.mongodb.net:27017', 'mmdev-shard-00-01.z7q8g.mongodb.net:27017', 'mmdev-shard-00-00.z7q8g.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-b4um4g-shard-0', tls=True, tlsallowinvalidcertificates=True, tlsdisableocspendpointcheck=True), 'ca_foundation'), 'CA-FO-P1')
Total Documents before Ingestion: 1949
Total Documents to be Ingested: 25
JSON file loaded successfully from: ./generated_embeddings/CA-FO-P4//CA-FO-P4-C9-U4 Exchange Rate and Its E

# MongoDB Vector Search Pipeline

In [28]:
try:
    # Retrieve a list of search indexes
    indexes = list(coll.list_search_indexes())
    print("Existing search indexes:")
    # Iterate through the list and print each index
    for index in indexes:
        print(index)
except Exception as e:
    print(f"Error listing search indexes: {e}")

Existing search indexes:
{'id': '67860feb2f646f39728ba02d', 'name': 'ca_foundation_index', 'type': 'vectorSearch', 'status': 'READY', 'queryable': True, 'latestDefinitionVersion': {'version': 0, 'createdAt': datetime.datetime(2025, 1, 14, 7, 19, 7, 409000)}, 'latestDefinition': {'fields': [{'type': 'vector', 'path': 'embedding', 'similarity': 'dotProduct', 'numDimensions': 3072}, {'type': 'filter', 'path': 'paper'}, {'type': 'filter', 'path': 'chapter'}, {'type': 'filter', 'path': 'unit'}]}, 'statusDetail': [{'hostname': 'atlas-b4um4g-shard-00-01', 'status': 'READY', 'queryable': True, 'mainIndex': {'status': 'READY', 'queryable': True, 'definitionVersion': {'version': 0, 'createdAt': datetime.datetime(2025, 1, 14, 7, 19, 7)}, 'definition': {'fields': [{'type': 'vector', 'path': 'embedding', 'numDimensions': 3072, 'similarity': 'dotProduct'}, {'type': 'filter', 'path': 'paper'}, {'type': 'filter', 'path': 'chapter'}, {'type': 'filter', 'path': 'unit'}]}}}, {'hostname': 'atlas-b4um4g-sh

## Create Vector Search Index

In [48]:
# from pymongo.operations import SearchIndexModel

# vector_search_index = "ca_foundation_index"
# # Define search index model
# search_index_model = SearchIndexModel(
#     definition={
#         "fields": [
#             {
#                 "type": "vector",
#                 "path": "embedding",
#                 "similarity": "dotProduct",
#                 "numDimensions": 3072,
#             },
#             {
#                 "type": "filter",
#                 "path": "paper",
#             },
#             {
#                 "type": "filter",
#                 "path": "chapter",
#             },
#             {
#                 "type": "filter",
#                 "path": "unit",
#             },
#         ]
#     },
#     name=vector_search_index,
#     type="vectorSearch",
# )

# coll.create_search_index(model=search_index_model)
# print("Vector search index created successfully.")


Vector search index created successfully.


## Retriever

In [29]:
coll

Collection(Database(MongoClient(host=['mmdev-shard-00-02.z7q8g.mongodb.net:27017', 'mmdev-shard-00-01.z7q8g.mongodb.net:27017', 'mmdev-shard-00-00.z7q8g.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-b4um4g-shard-0', tls=True, tlsallowinvalidcertificates=True, tlsdisableocspendpointcheck=True), 'ca_foundation'), 'CA-FO-P1')

In [67]:
from langchain_mongodb import MongoDBAtlasVectorSearch

vector_search_index = "ca_foundation_index"

vector_search = MongoDBAtlasVectorSearch(
    collection=coll,
    embedding=embeddingmodel_init(),
    index_name=vector_search_index,
)

vector_search.similarity_search_with_score(
    query="DETERMINATION OF PRICES",
    k=10,
    pre_filter={
        "paper": "CA-FO-P4",
        # "chapter": "CA-IN-P5-C4-AUDIT EVIDENCE",
        # "unit": "CA-IN-P5-C4-AUDIT EVIDENCE",
    },
    post_filter_pipeline=[],
)


Embedding model: client=<openai.resources.embeddings.Embeddings object at 0x1155aa600> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x115527c80> model='text-embedding-3-large' dimensions=3072 deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=() chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True


[(Document(metadata={'_id': '67e64bfd33da581f427087fa', 'paper': 'CA-FO-P4', 'chapter': 'CA-FO-P4-C4-U2-DETERMINATION OF PRICES', 'unit': 'CA-FO-P4-C4-U2-DETERMINATION OF PRICES'}, page_content='INTRODUCTION OF DETERMINATION OF PRICES\nPrices of goods express their exchange value. Prices are also used for expressing the value of various services rendered by different factors of production such as land, labour, capital and organization in the form of rent, wages, interest and profit respectively. Therefore, the concept of price, especially the process of price determination, is of vital importance in Economics.\nIn this unit, we shall learn how demand and supply interact to strike a balance so that equilibrium price is determined in a free market. A free market is one in which the forces of demand and supply are free to take their own course and there is no intervention from outside by government or any other entity. It is to be noted that, generally, it is the interaction between deman

In [71]:
from langchain_mongodb import MongoDBAtlasVectorSearch

vector_search_index = "ca_foundation_index"

vector_search = MongoDBAtlasVectorSearch(
    collection=coll,
    embedding=embeddingmodel_init(),
    index_name=vector_search_index,
)

vector_search.similarity_search_with_score(
    query="Explain me about Accounting",
    k=10,
    pre_filter={
        "chapter": "CA-FO-P1-C2",
        "unit": "CA-FO-P1-C2-U1-Basic Accounting Procedures-Journal Entries",
    },
    post_filter_pipeline=[],
)

Embedding model: client=<openai.resources.embeddings.Embeddings object at 0x115194140> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x115195df0> model='text-embedding-3-large' dimensions=3072 deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=() chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True


[]

In [98]:
def embeddingmodel_init():
    import os
    from dotenv import load_dotenv
    from langchain_openai import OpenAIEmbeddings

    load_dotenv()

    OPENAI_API_KEY: str | None = os.getenv(key="OPENAI_API_KEY", default=None)

    embedding_model = OpenAIEmbeddings(
        model="text-embedding-3-large",
        dimensions=3072,
        disallowed_special=(),
        api_key=OPENAI_API_KEY,
    )
    print(f"Embedding model: {embedding_model}")
    return embedding_model


def mongodb_init():
    import os
    from dotenv import load_dotenv
    from pymongo import MongoClient

    load_dotenv()

    MONGODB_CONNECTION_STRING: str | None = os.getenv(
        key="MONGODB_CONNECTION_STRING", default=None
    )

    client = MongoClient(
        host=MONGODB_CONNECTION_STRING,
        tls=True,
        tlsAllowInvalidCertificates=True,
    )
    print(f"MongoDB client: {client}")
    return client


def mongodb_retriever_pipeline(
    query: str = "Explain me about Accounting",
    vector_search_index: str = "ca_foundation_index",
    db_name: str = "ca_foundation",
    collectoin_name: str = "CA-FO-P1",
):
    from langchain_mongodb import MongoDBAtlasVectorSearch

    client = mongodb_init()
    db = client.get_database(name=db_name)
    coll = db.get_collection(name=collectoin_name)
    print(f"Collection: {coll}")

    vector_search = MongoDBAtlasVectorSearch(
        collection=coll,
        embedding=embeddingmodel_init(),
        index_name=vector_search_index,
    )
    result = vector_search.similarity_search_with_score(
        query=query,
        k=10,
        pre_filter={},
        post_filter_pipeline=[],
    )
    return result


mongodb_retriever_pipeline(query="Explain me about Dissolution of Partnership.")

MongoDB client: MongoClient(host=['mmdev-shard-00-01.z7q8g.mongodb.net:27017', 'mmdev-shard-00-00.z7q8g.mongodb.net:27017', 'mmdev-shard-00-02.z7q8g.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-b4um4g-shard-0', tls=True, tlsallowinvalidcertificates=True, tlsdisableocspendpointcheck=True)
Collection: Collection(Database(MongoClient(host=['mmdev-shard-00-01.z7q8g.mongodb.net:27017', 'mmdev-shard-00-00.z7q8g.mongodb.net:27017', 'mmdev-shard-00-02.z7q8g.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-b4um4g-shard-0', tls=True, tlsallowinvalidcertificates=True, tlsdisableocspendpointcheck=True), 'ca_foundation'), 'CA-FO-P1')
Embedding model: client=<openai.resources.embeddings.Embeddings object at 0x114af49e0> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x114af6420> model='text-e

[(Document(metadata={'_id': '67860acaea751353baadcdca', 'paper': 'CA-FO-P1', 'chapter': 'CA-FO-P1-C10', 'unit': 'CA-FO-P1-C10-U6- Dissolution of Partnership Firms and LLP'}, page_content="Introduction of Dissolution of Partnership Firms and LLP\nApart from the readjustment of rights of partners in the share of profit by way of change in the profit-sharing ratio and admission of a new partner or for retirement/death of a partner, another important aspect of partnership accounts is how to close books of accounts in case of dissolution. In this Unit, we will discuss the circumstances leading to the dissolution of a partnership firm and accounting treatment necessary to close its books of accounts. Also, we will discuss the special problems relating to the insolvency of partners and the settlement of the partnership's liabilities."),
  0.8384476900100708),
 (Document(metadata={'_id': '67860acaea751353baadcdcb', 'paper': 'CA-FO-P1', 'chapter': 'CA-FO-P1-C10', 'unit': 'CA-FO-P1-C10-U6- Disso